In [1]:
%matplotlib widget

In [2]:
import numpy as np
np.set_printoptions(linewidth=100)
import sympy as sm
import sympy.abc as sbl
from scipy.sparse import diags
from scipy.integrate import RK45
import scipy.sparse.linalg as la
import scipy.sparse as sp
sm.init_printing()
import time
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import HBox, IntSlider

plt.ioff()
plt.clf()

## Links to important Research Papers


<h3 align="center" style="color:#FFE647;">Equation   </h3> 
\begin{equation}
\\  A \alpha^{'} +\hspace{15mm}\beta ( D \alpha \hspace{10mm} - \hspace{10mm}E \alpha) \hspace{10mm} - \beta F  \alpha \hspace{15mm} = \hspace{15mm}f(t)
\end{equation}


\begin{equation}
\\  \alpha^{'} = -A^{-1}(\beta ( D \alpha  - E \alpha) - \beta F  \alpha - f(t)) \hspace{10mm} (i)
\\  \alpha^{'} = -A^{-1}( \beta ( D \alpha  - E \alpha -  F  \alpha )- f(t)) \hspace{10mm} (i)
\end{equation}



In [3]:
class Matrices:
    
    
    def __init__(self, N=10):
        
        
        self.N = N
        self.shape = (N+1, N+1)
        self.h = 1/N
        self.domain = np.linspace(0, 1, N+1) 
        self.beta = -1
        self.time_step = self.h**2
        self.runs = 1
        
        self.A = sp.csc_matrix(diags([       [self.h/6]*(N),   \
                                [self.h/3, *[2*self.h/3]*(N-1), self.h/3], \
                                     [self.h/6]*(N)],               [1, 0, -1] ), dtype=np.float32)

        self.B = np.array([self.matB(i) for i in range(N+1)]).reshape(N+1, 1) 

        self.C = sp.csr_matrix(diags([[1/2]*(N), [-1/2]+[0]*(N-1)+[1/2], [-1/2]*(N)] , [1, 0, -1]), dtype=np.float32)

        self.D = sp.csr_matrix(([-1/self.h, 1/self.h],([N, N-1], [N]*2)),self.shape,  dtype=np.float32)

        self.E = sp.csr_matrix( ([-1/self.h, 1/self.h],([0, 0], [0, 1])), self.shape,  dtype=np.float32)

        self.F = sp.csr_matrix(diags([      [-1/self.h]*(N),  
                                     [1/self.h, *[2/self.h]*(N-1), 1/
                                      
                                      self.h], \
                                          [-1/self.h]*(N)   ],       [1, 0, -1] ), dtype=np.float32)


        
        #Set initial conditions here
        self.alpha = np.sin(np.pi* self.domain).reshape(N+1, 1)
        
        #Set solution here
        self.exact = lambda t: np.exp(t)*np.sin(np.pi * self.domain)
        

    

    
    def matB(self,i):
            if i == 0 :
                row = np.array([0, 0, 1, 1])
                col = np.array([0, 1, 0, 1])
                data = np.array([-1/3, 1/3, -1/6, 1/6])
                return sp.csr_matrix((data, (row, col)), shape=self.shape,dtype=np.float32)        
            elif i == self.N:
                row = np.array([self.N-1, self.N - 1, self.N, self.N])
                col = np.array([self.N-1, self.N , self.N -1, self.N])
                data = np.array([-1/6, 1/6, -1/3, 1/3])
                return sp.csr_matrix((data, (row, col)), shape=self.shape, dtype=np.float32)   
            else:
                row = np.array([i-1]*3 + [i]*3 + [i+1]*3)
                col = np.array([col for col in range(i-1, i+1 + 1)]*3, dtype=np.float32)
                data = np.array([-1/6, 1/6, 0, -1/3, 0, 1/3, 0, -1/6, 1/6 ])
                return sp.csr_matrix((data, (row, col)), shape=self.shape, dtype=np.float32)    
            
   
    

In [4]:
class backward_euler(Matrices):
    
    def __init__(self, num_basis = 10):
     
        super().__init__(int(num_basis))
        
        self.pos = lambda k: ((1 + np.pi**2)*(-np.pi*self.h*np.cos(np.pi*self.h*k) \
                            + np.sin(np.pi*self.h*k) - np.sin(np.pi*self.h*(k - 1)))/(np.pi**2*self.h))
        self.neg = lambda k: ((1 + np.pi**2)*(np.pi*self.h*np.cos(np.pi*self.h*k) +\
                            np.sin(np.pi*self.h*k) - np.sin(np.pi*self.h*(k + 1)))/(np.pi**2*self.h))
        self.f_without_t = (  np.array([0]+[self.pos(k) for k in range(1, self.N+1)]) +\
                            np.array([self.neg(k) for k in range(0, self.N)]+[0]) \
                            ).reshape(self.N+1, 1)

        #Backward Euler

        self.time_point = 0
        
        for i in range(self.runs):
            self.time_point += self.time_step  
            self.alpha[0], self.alpha[-1] = (0, 0)
            
            f_t = np.exp(self.time_point)* self.f_without_t

            self.alpha = la.inv(self.A-self.time_step * self.beta *(self.D - self.E - self.F)).dot\
                                (self.A.dot(self.alpha) + self.time_step * f_t)  
            
            
        self.approx = self.alpha.ravel()  

    def graph(self):
        plt.legend(["Approx", "Exact"])   


        return plt.plot(self.domain, self.approx, self.domain, self.exact(self.time_point))

In [5]:

class rk45(Matrices): 
    
    def __init__(self, num_basis = 10):#, method = "backward_euler"):

        super().__init__(num_basis)

        self.pos = lambda k: ((1 + np.pi**2)*(-np.pi*self.h*np.cos(np.pi*self.h*k) \
                            + np.sin(np.pi*self.h*k) - np.sin(np.pi*self.h*(k - 1)))/(np.pi**2*self.h))
        self.neg = lambda k: ((1 + np.pi**2)*(np.pi*self.h*np.cos(np.pi*self.h*k) +\
                            np.sin(np.pi*self.h*k) - np.sin(np.pi*self.h*(k + 1)))/(np.pi**2*self.h))
        self.f_without_t = (  np.array([0]+[self.pos(k) for k in range(1, self.N+1)]) +\
                            np.array([self.neg(k) for k in range(0, self.N)]+[0]) \
                            ).reshape(self.N+1, 1)
        

       #Runga Kutta 4(5)th order 

        self.time_point = 0
        
        def fun(t, y):
            f_t = np.exp(t)* self.f_without_t
            y[0] = 0
            y[-1] = 0
            
            return -la.inv(self.A).dot(self.beta*((self.D - self.E - self.F).dot(y) - f_t))

        self.alpha = RK45(fun,self.time_point,  self.alpha.ravel(), first_step=self.h**2, \
                          t_bound=self.h**2, vectorized=True ).y    

        
           
        self.approx = self.alpha

    def graph(self):
        plt.legend(["Approx", "Exact"]) 
        

In [6]:
def graphit(string = "rk45"):


        slider = IntSlider(
            orientation='vertical',
            value=10,
            min=10,
            max=1000,
            step=1
        )

        fig = plt.figure(3)

        heateqn = eval(string +"(slider.value)")

        lines = plt.plot(heateqn.domain, heateqn.approx, "r")
        def update_lines(change):
            heateqn = backward_euler(slider.value)

            lines[0].set_data(heateqn.domain, heateqn.approx)
            fig.canvas.draw()
            fig.canvas.flush_events()

        slider.observe(update_lines, names='value')

        return HBox([slider, fig.canvas])


In [7]:
graphit()
